# Sudoku Autocomplete BFS - Graph Search Algorithm

In [1]:
from queue import Queue
import copy
import time

In [2]:
class Problem(object):
    def __init__(self, initial):
        self.initial = initial
        self.type = len(initial) # Defines board type, either 6x6 or 9x9
        self.height = int(self.type/3) # Defines height of quadrant (2 for 6x6, 3 for 9x9)

    # Return set of valid numbers from values that do not appear in used
    def filter_values(self, possible_values, used):
        return [number for number in possible_values if number not in used]

    # Return first empty spot on grid (marked with 0)
    def get_spot(self, board, state):
        for row in range(board):
            for column in range(board):
                if state[row][column] == 0:
                    return row, column   

    def actions(self, state):
        number_set = range(1, self.type+1) # Defines set of valid numbers that can be placed on board
        in_column = [] # List of valid values in spot's column
        in_block = [] # List of valid values in spot's quadrant

        row,column = self.get_spot(self.type, state) # Get first empty spot on board

        # Filter valid values based on row
        in_row = [number for number in state[row] if (number != 0)]
        options = self.filter_values(number_set, in_row)

        # Filter valid values based on column
        for column_index in range(self.type):
            if state[column_index][column] != 0:
                in_column.append(state[column_index][column])
        options = self.filter_values(options, in_column)

        # Filter with valid values based on quadrant
        row_start = int(row/self.height)*self.height
        column_start = int(column/3)*3
        
        for block_row in range(0, self.height):
            for block_column in range(0,3):
                in_block.append(state[row_start + block_row][column_start + block_column])
        options = self.filter_values(options, in_block)

        for number in options:
            yield number, row, column      

    # Returns updated board after adding new valid value
    def result(self, state, action):
        play = action[0]
        row = action[1]
        column = action[2]

        # Add new valid value to board
        new_state = copy.deepcopy(state)
        new_state[row][column] = play

        return new_state

    # Use sums of each row, column and quadrant to determine validity of board state
    def goal_test(self, state):
        # Expected sum of each row, column or quadrant.
        total = sum(range(1, self.type+1))

        # Check rows and columns and return false if total is invalid
        for row in range(self.type):
            if (len(state[row]) != self.type) or (sum(state[row]) != total):
                return False

            column_total = 0
            for column in range(self.type):
                column_total += state[column][row]

            if (column_total != total):
                return False

        # Check quadrants and return false if total is invalid
        for column in range(0,self.type,3):
            for row in range(0,self.type,self.height):

                block_total = 0
                for block_row in range(0,self.height):
                    for block_column in range(0,3):
                        block_total += state[row + block_row][column + block_column]

                if (block_total != total):
                    return False

        return True

In [3]:
class Node:

    def __init__(self, state, action=None):
        self.state = state
        self.action = action

    # Use each action to create a new board state
    def expand(self, problem):
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    # Return node with new board state
    def child_node(self, problem, action):
        next = problem.result(self.state, action)
        return Node(next, action)

In [4]:
def BFS(problem):
    # Create initial node of problem tree holding original board
    node = Node(problem.initial)
    # Check if original board is correct and immediately return if valid
    if problem.goal_test(node.state):
        return node

    frontier = Queue()
    frontier.put(node)

    # Loop until all nodes are explored or solution found
    while (frontier.qsize() != 0):

        node = frontier.get()
        for child in node.expand(problem):
            if problem.goal_test(child.state):
                return child

            frontier.put(child)

    return None

In [30]:
def solve_bfs(board):
    print ("\nSolving with BFS...")
    start_time = time.time()
    
    switcher = {
        0: "          ",
        1: "      \   ",
        2: "       \  ",
        3: "--------\ ",
        4: "         >",
        5: "--------/ ",
        6: "       /  ",
        7: "      /   ",
        8: "          ",
    }

    problem = Problem(board)
    solution = BFS(problem)
    elapsed_time = time.time() - start_time

    if solution:
        print ("Found solution")
        for i, row in enumerate(solution.state):
            print (str(board[i]) + switcher.get(i) + str(row))
    else:
        print ("No possible solutions")

    print ("Elapsed time: " + str(elapsed_time))

In [41]:
#Testing on invalid 9x9 board...
board = [[0,0,9,0,7,0,0,0,5],
      [0,0,2,1,0,0,9,0,0],
      [1,0,0,0,2,8,0,0,0],
      [0,7,0,0,0,5,0,0,1],
      [0,0,8,5,1,0,0,0,0],
      [0,5,0,0,0,0,3,0,0],
      [0,0,0,0,0,3,0,0,6],
      [8,0,0,0,0,0,0,0,0],
      [2,1,0,0,0,0,0,8,7]]

print ("Problem:")
for row in board:
      print (row)

Problem:
[0, 0, 9, 0, 7, 0, 0, 0, 5]
[0, 0, 2, 1, 0, 0, 9, 0, 0]
[1, 0, 0, 0, 2, 8, 0, 0, 0]
[0, 7, 0, 0, 0, 5, 0, 0, 1]
[0, 0, 8, 5, 1, 0, 0, 0, 0]
[0, 5, 0, 0, 0, 0, 3, 0, 0]
[0, 0, 0, 0, 0, 3, 0, 0, 6]
[8, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 1, 0, 0, 0, 0, 0, 8, 7]


In [42]:
solve_bfs(board)


Solving with BFS...
No possible solutions
Elapsed time: 1.1669952869415283


In [44]:
#Testing on valid 9x9 board...
board = [[0,0,0,8,4,0,6,5,0],
      [0,8,0,0,0,0,0,0,9],
      [0,0,0,0,0,5,2,0,1],
      [0,3,4,0,7,0,5,0,6],
      [0,6,0,2,5,1,0,3,0],
      [5,0,9,0,6,0,7,2,0],
      [1,0,8,5,0,0,0,0,0],
      [6,0,0,0,0,0,0,4,0],
      [0,5,2,0,8,6,0,0,0]]
print ("Problem:")
for row in board:
      print (row)

Problem:
[0, 0, 0, 8, 4, 0, 6, 5, 0]
[0, 8, 0, 0, 0, 0, 0, 0, 9]
[0, 0, 0, 0, 0, 5, 2, 0, 1]
[0, 3, 4, 0, 7, 0, 5, 0, 6]
[0, 6, 0, 2, 5, 1, 0, 3, 0]
[5, 0, 9, 0, 6, 0, 7, 2, 0]
[1, 0, 8, 5, 0, 0, 0, 0, 0]
[6, 0, 0, 0, 0, 0, 0, 4, 0]
[0, 5, 2, 0, 8, 6, 0, 0, 0]


In [45]:
solve_bfs(board)


Solving with BFS...
Found solution
[0, 0, 0, 8, 4, 0, 6, 5, 0]          [7, 2, 1, 8, 4, 9, 6, 5, 3]
[0, 8, 0, 0, 0, 0, 0, 0, 9]      \   [3, 8, 5, 6, 1, 2, 4, 7, 9]
[0, 0, 0, 0, 0, 5, 2, 0, 1]       \  [9, 4, 6, 7, 3, 5, 2, 8, 1]
[0, 3, 4, 0, 7, 0, 5, 0, 6]--------\ [2, 3, 4, 9, 7, 8, 5, 1, 6]
[0, 6, 0, 2, 5, 1, 0, 3, 0]         >[8, 6, 7, 2, 5, 1, 9, 3, 4]
[5, 0, 9, 0, 6, 0, 7, 2, 0]--------/ [5, 1, 9, 4, 6, 3, 7, 2, 8]
[1, 0, 8, 5, 0, 0, 0, 0, 0]       /  [1, 7, 8, 5, 9, 4, 3, 6, 2]
[6, 0, 0, 0, 0, 0, 0, 4, 0]      /   [6, 9, 3, 1, 2, 7, 8, 4, 5]
[0, 5, 2, 0, 8, 6, 0, 0, 0]          [4, 5, 2, 3, 8, 6, 1, 9, 7]
Elapsed time: 0.25800156593322754


In [46]:
#Testing on filled valid 9x9 board...
board = [[9,7,4,2,3,6,1,5,8],
      [6,3,8,5,9,1,7,4,2],
      [1,2,5,4,8,7,9,3,6],
      [3,1,6,7,5,4,2,8,9],
      [7,4,2,9,1,8,5,6,3],
      [5,8,9,3,6,2,4,1,7],
      [8,6,7,1,2,5,3,9,4],
      [2,5,3,6,4,9,8,7,1],
      [4,9,1,8,7,3,6,2,5]]
print ("Problem:")
for row in board:
      print (row)

Problem:
[9, 7, 4, 2, 3, 6, 1, 5, 8]
[6, 3, 8, 5, 9, 1, 7, 4, 2]
[1, 2, 5, 4, 8, 7, 9, 3, 6]
[3, 1, 6, 7, 5, 4, 2, 8, 9]
[7, 4, 2, 9, 1, 8, 5, 6, 3]
[5, 8, 9, 3, 6, 2, 4, 1, 7]
[8, 6, 7, 1, 2, 5, 3, 9, 4]
[2, 5, 3, 6, 4, 9, 8, 7, 1]
[4, 9, 1, 8, 7, 3, 6, 2, 5]


In [47]:
solve_bfs(board)


Solving with BFS...
Found solution
[9, 7, 4, 2, 3, 6, 1, 5, 8]          [9, 7, 4, 2, 3, 6, 1, 5, 8]
[6, 3, 8, 5, 9, 1, 7, 4, 2]      \   [6, 3, 8, 5, 9, 1, 7, 4, 2]
[1, 2, 5, 4, 8, 7, 9, 3, 6]       \  [1, 2, 5, 4, 8, 7, 9, 3, 6]
[3, 1, 6, 7, 5, 4, 2, 8, 9]--------\ [3, 1, 6, 7, 5, 4, 2, 8, 9]
[7, 4, 2, 9, 1, 8, 5, 6, 3]         >[7, 4, 2, 9, 1, 8, 5, 6, 3]
[5, 8, 9, 3, 6, 2, 4, 1, 7]--------/ [5, 8, 9, 3, 6, 2, 4, 1, 7]
[8, 6, 7, 1, 2, 5, 3, 9, 4]       /  [8, 6, 7, 1, 2, 5, 3, 9, 4]
[2, 5, 3, 6, 4, 9, 8, 7, 1]      /   [2, 5, 3, 6, 4, 9, 8, 7, 1]
[4, 9, 1, 8, 7, 3, 6, 2, 5]          [4, 9, 1, 8, 7, 3, 6, 2, 5]
Elapsed time: 0.0


In [48]:
# Testing on valid 9x9 board...
board = [[3,0,5,4,2,0,8,1,0],
      [4,8,7,9,0,1,5,0,6],
      [0,2,9,0,5,6,3,7,4],
      [8,5,0,7,9,3,0,4,1],
      [6,1,3,2,0,8,9,5,7],
      [0,7,4,0,6,5,2,8,0],
      [2,4,1,3,0,9,0,6,5],
      [5,0,8,6,7,0,1,9,2],
      [0,9,6,5,1,2,4,0,8]]
print ("Problem:")
for row in board:
      print (row)

Problem:
[3, 0, 5, 4, 2, 0, 8, 1, 0]
[4, 8, 7, 9, 0, 1, 5, 0, 6]
[0, 2, 9, 0, 5, 6, 3, 7, 4]
[8, 5, 0, 7, 9, 3, 0, 4, 1]
[6, 1, 3, 2, 0, 8, 9, 5, 7]
[0, 7, 4, 0, 6, 5, 2, 8, 0]
[2, 4, 1, 3, 0, 9, 0, 6, 5]
[5, 0, 8, 6, 7, 0, 1, 9, 2]
[0, 9, 6, 5, 1, 2, 4, 0, 8]


In [49]:
solve_bfs(board)


Solving with BFS...
Found solution
[3, 0, 5, 4, 2, 0, 8, 1, 0]          [3, 6, 5, 4, 2, 7, 8, 1, 9]
[4, 8, 7, 9, 0, 1, 5, 0, 6]      \   [4, 8, 7, 9, 3, 1, 5, 2, 6]
[0, 2, 9, 0, 5, 6, 3, 7, 4]       \  [1, 2, 9, 8, 5, 6, 3, 7, 4]
[8, 5, 0, 7, 9, 3, 0, 4, 1]--------\ [8, 5, 2, 7, 9, 3, 6, 4, 1]
[6, 1, 3, 2, 0, 8, 9, 5, 7]         >[6, 1, 3, 2, 4, 8, 9, 5, 7]
[0, 7, 4, 0, 6, 5, 2, 8, 0]--------/ [9, 7, 4, 1, 6, 5, 2, 8, 3]
[2, 4, 1, 3, 0, 9, 0, 6, 5]       /  [2, 4, 1, 3, 8, 9, 7, 6, 5]
[5, 0, 8, 6, 7, 0, 1, 9, 2]      /   [5, 3, 8, 6, 7, 4, 1, 9, 2]
[0, 9, 6, 5, 1, 2, 4, 0, 8]          [7, 9, 6, 5, 1, 2, 4, 3, 8]
Elapsed time: 0.003000020980834961


In [50]:
# Testing unsolvable quadrant on a 9x9 board...
board = [[0,9,0,3,0,0,0,0,1],
      [0,0,0,0,8,0,0,4,6],
      [0,0,0,0,0,0,8,0,0],
      [4,0,5,0,6,0,0,3,0],
      [0,0,3,2,7,5,6,0,0],
      [0,6,0,0,1,0,9,0,4],
      [0,0,1,0,0,0,0,0,0],
      [5,8,0,0,2,0,0,0,0],
      [2,0,0,0,0,7,0,6,0]]
print ("Problem:")
for row in board:
      print (row)

Problem:
[0, 9, 0, 3, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 8, 0, 0, 4, 6]
[0, 0, 0, 0, 0, 0, 8, 0, 0]
[4, 0, 5, 0, 6, 0, 0, 3, 0]
[0, 0, 3, 2, 7, 5, 6, 0, 0]
[0, 6, 0, 0, 1, 0, 9, 0, 4]
[0, 0, 1, 0, 0, 0, 0, 0, 0]
[5, 8, 0, 0, 2, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 7, 0, 6, 0]


In [51]:
solve_bfs(board)


Solving with BFS...
No possible solutions
Elapsed time: 0.9689986705780029
